# OVerview

Just update concepts.txt, then run all. It will had 5 questions and answers to db. 

# Load concepts

In [1]:
with open('concepts.txt', 'r') as f:
    concepts = [line.strip() for line in f if line.strip()]
for c in concepts:
    print(c)

Fact Table Dimensional Modelling
Fact Table
Dimension Table
L1 Cache
L2 Cache
Register
XGboost
decision tree
backpropogation
mean squared error
mean absolute error
root mean squared error


# For each concept, if doesn't have N qa records, query more 

In [2]:
from backend.connections import get_postgres_db

from backend.crud import create_concept, create_question
from sqlalchemy import text 

from backend import models 
from backend.llm_interface.InterfaceCreateQA import InterfaceCreateQA

QA_PROMPT_TYPE = 'generate_qa'


def check_if_concept_has_qa(concept):
    concept_object = next(get_postgres_db()).query(models.ConceptStore).filter(models.ConceptStore.concept == concept).first()
    concept_id = concept_object.concept_id
    questions_for_concept = next(get_postgres_db()).query(models.Question).filter(models.Question.concept_id == concept_id).all()
    if len(questions_for_concept) > 0:
        
        return True
    return False

def add_questions_for_concept_if_none_exist(concept):
    if check_if_concept_has_qa(concept):
        print(f'{concept} has questions')
    else:
        
        # get concept id 
        concept_object = next(get_postgres_db()).query(models.ConceptStore).filter(models.ConceptStore.concept == concept).first()
        concept_id = concept_object.concept_id
        print(f'creating qa for {concept}, with concept id = {concept_id}')

        # get prompt id 
        prompt_record = (
            next(get_postgres_db()).query(models.PromptStore)
            .filter(models.PromptStore.prompt_type == QA_PROMPT_TYPE)
            .order_by(models.PromptStore.created_date.desc())
            .first()
        )
        print(f' using prompt id = {prompt_record.prompt_id}')
        
        # create questions 
        qa_interface = InterfaceCreateQA(prompt_id=prompt_record.prompt_id, prompt=prompt_record.prompt)
        response = qa_interface.create_quiz_for_concept(concept)
        for question_number, question_answer in response.items():
                create_question(
                    next(get_postgres_db()), 
                    question_answer['question'], 
                    question_answer['answer'], 
                    concept_id=concept_id,
                    prompt_id=prompt_record.prompt_id
                )
                print(f'    created : q={question_answer["question"]}, a={question_answer["answer"]}')


for c in concepts:
     add_questions_for_concept_if_none_exist(c)

Fact Table Dimensional Modelling has questions
Fact Table has questions
Dimension Table has questions
L1 Cache has questions
L2 Cache has questions
Register has questions
XGboost has questions
decision tree has questions
backpropogation has questions
mean squared error has questions
creating qa for mean absolute error, with concept id = 361362c1-db45-46f3-ad83-9a0db6120f5d
 using prompt id = 25d1a107-44ae-45dc-a79a-2de72c47eff7
    created : q=What is mean absolute error (MAE) and how is it calculated?, a=Mean Absolute Error (MAE) is a measure of the average magnitude of errors in a set of predictions, without considering their direction. It is calculated by taking the average of the absolute differences between predicted and actual values, expressed as MAE = (1/n) * Σ|actual - predicted|, where n is the number of observations.
    created : q=Why is MAE considered an important metric in regression analysis?, a=MAE is important in regression analysis because it provides a clear underst